In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import cv2
from tqdm import tqdm
from torchsummary import summary
from fvcore.nn import FlopCountAnalysis, parameter_count
from ptflops import get_model_complexity_info
import time
from torch.fx import symbolic_trace

c:\Users\PARK\anaconda3\envs\eai\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Device 설정
device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
# 데이터셋 경로 설정
data_dir = "data"

In [4]:
# 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

In [5]:
# 데이터셋 로드
dataset = ImageFolder(root=data_dir, transform=transform)
print(f"Classes: {dataset.classes}")

Classes: ['with_mask', 'without_mask']


In [6]:
# Train:Val:Test = 70:15:15 분할
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

In [7]:
# DataLoader 생성
batch_size = 16
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [8]:
class MaskClassifier(nn.Module):
    def __init__(self):
        super(MaskClassifier, self).__init__()
        
        # Feature Extraction - 더 얕은 구조로 변경
        self.features = nn.Sequential(
            # First Block
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
            
            # Second Block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
            
            # Third Block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
        )
        
        # Classifier - 더 단순한 구조로 변경
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [9]:
# 기존 모델 로드
model = MaskClassifier().to(device)
model.load_state_dict(torch.load("mask_classifier.pth"))

<All keys matched successfully>

In [ ]:
# def apply_pruning(model, amount=0.3):
#     # 먼저 프루닝 적용
#     for name, module in model.named_modules():
#         if isinstance(module, nn.Conv2d):
#             prune.l1_unstructured(module, name='weight', amount=amount)
#         elif isinstance(module, nn.Linear):
#             prune.l1_unstructured(module, name='weight', amount=amount)
#     return model

def apply_pruning(model, amount=0.5):
        """모델의 구조를 유지하면서 프루닝만 적용"""
        for name, module in model.named_modules():
            if isinstance(module, (nn.Conv2d, nn.Linear)):
                prune.l1_unstructured(module, name='weight', amount=amount)
                prune.remove(module, 'weight')
        return model

In [ ]:
# def make_pruning_permanent(model):
#     # 프루닝을 영구적으로 적용
#     for name, module in model.named_modules():
#         if isinstance(module, (nn.Conv2d, nn.Linear)):
#             try:
#                 prune.remove(module, 'weight')
#             except:
#                 pass
#     return model

In [12]:
# Pruning 적용
print("Applying pruning...")
apply_pruning(model, amount=0.3)

Applying pruning...


MaskClassifier(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout2d(p=0.2, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout2d(p=0.2, inplace=False)
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Dropout2d(p=0.2, inplace=False)
 

In [13]:
# Loss function과 optimizer 설정
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
num_epoch = 20
optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
    betas=(0.9, 0.999),
    weight_decay=0.0001
)

In [14]:
def train_model(model, train_loader, val_loader, epochs=10):
    best_val_acc = 0.0
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", unit="batch")
        
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Progress Bar에 현재 배치의 accuracy 표시
            batch_acc = 100. * correct / total
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{batch_acc:.2f}%'
            })
    
    # 최종 학습 결과 평가
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    final_acc = 100. * correct / total
    print("\n=== Final Training Results ===")
    print(f"Final Validation Accuracy: {final_acc:.2f}%")

In [15]:
# 모델 학습 실행
train_model(model, train_loader, val_loader, num_epoch)

Epoch 20/20: 100%|██████████| 144/144 [02:09<00:00,  1.11batch/s, loss=0.4266, acc=94.46%]



=== Final Training Results ===
Final Validation Accuracy: 95.93%


In [16]:
# 테스트 함수
def test_model(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc="Testing", unit="batch")  # Progress Bar 추가
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            
            progress_bar.set_postfix(batch_accuracy=(preds == labels).float().mean().item())
            
    print("\nTest Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=dataset.classes))

In [17]:
# 테스트 실행
test_model(model, test_loader)

Testing: 100%|██████████| 31/31 [00:19<00:00,  1.57batch/s, batch_accuracy=1]    


Test Classification Report:
              precision    recall  f1-score   support

   with_mask       0.93      0.97      0.95       232
without_mask       0.97      0.93      0.95       260

    accuracy                           0.95       492
   macro avg       0.95      0.95      0.95       492
weighted avg       0.95      0.95      0.95       492



In [ ]:
# # 프루닝 영구 적용
# print("\nMaking pruning permanent...")
# make_pruning_permanent(model)


Making pruning permanent...


MaskClassifier(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout2d(p=0.2, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout2d(p=0.2, inplace=False)
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Dropout2d(p=0.2, inplace=False)
 

In [19]:
# Pruned 모델 저장
def save_pruned_model(model, path="pruned_mask_classifier03.pth"):
    torch.save(model.state_dict(), path)
    print(f"Pruned model saved to {path}")

In [20]:
save_pruned_model(model, "pruned_mask_classifier03.pth")

Pruned model saved to pruned_mask_classifier03.pth


In [21]:
def count_parameters(model):
    total_params = 0
    zero_params = 0
    for name, param in model.named_parameters():
        num_params = param.numel()
        total_params += num_params
        zero_params += (param == 0).sum().item()
        print(f"{name}: {num_params} parameters, {(param == 0).sum().item()} zeros")
    
    print(f"\nTotal parameters: {total_params:,}")
    print(f"Zero parameters: {zero_params:,}")
    print(f"Sparsity: {100 * zero_params / total_params:.2f}%")
    return total_params, zero_params

# Create and analyze original model
original_model = MaskClassifier().to(device)
print("Original Model Parameters:")
orig_total, orig_zeros = count_parameters(original_model)

# Load and analyze pruned model
pruned_model = MaskClassifier().to(device)
pruned_model.load_state_dict(torch.load("pruned_mask_classifier03.pth", map_location=device))
print("\nPruned Model Parameters:")
pruned_total, pruned_zeros = count_parameters(pruned_model)

Original Model Parameters:
features.0.weight: 864 parameters, 0 zeros
features.0.bias: 32 parameters, 0 zeros
features.1.weight: 32 parameters, 0 zeros
features.1.bias: 32 parameters, 32 zeros
features.5.weight: 18432 parameters, 0 zeros
features.5.bias: 64 parameters, 0 zeros
features.6.weight: 64 parameters, 0 zeros
features.6.bias: 64 parameters, 64 zeros
features.10.weight: 73728 parameters, 0 zeros
features.10.bias: 128 parameters, 0 zeros
features.11.weight: 128 parameters, 0 zeros
features.11.bias: 128 parameters, 128 zeros
classifier.2.weight: 256 parameters, 0 zeros
classifier.2.bias: 2 parameters, 0 zeros

Total parameters: 93,954
Zero parameters: 224
Sparsity: 0.24%

Pruned Model Parameters:
features.0.weight: 864 parameters, 259 zeros
features.0.bias: 32 parameters, 0 zeros
features.1.weight: 32 parameters, 0 zeros
features.1.bias: 32 parameters, 0 zeros
features.5.weight: 18432 parameters, 5530 zeros
features.5.bias: 64 parameters, 0 zeros
features.6.weight: 64 parameters,